# Cloud

## Dashboarding Frameworks

### Voila
* 可以做動態網頁，backed by Jupyter server，也可以佈署到 mybinder。用 ?urlpath=voila，參考 [Voila GitHub page](https://github.com/voila-dashboards/voila) 裡的 binder link
    * 直接指向一個 app 的 link 長這樣：https://mybinder.org/v2/gh/yc14e/nb2pdf/master?urlpath=voila/render/nb2pdf.ipynb
* 把 output cell 呈現出來。[xwidgets](https://github.com/jupyter-xeus/xwidgets) 應該也能呈現
    * 實測 HoloViz Panel 不太能 render，因為 Voila 遇到 JavaScript 會有很多問題
* 在 JupyterLab 上開發時 notebook 上方有 Render with Voila 按紐
* 有 [gridstack template](https://github.com/voila-dashboards/voila-gridstack) 可以控制 dashboard layout，但如果完全用 HoloViz Panel 做 dashboard 就不需要安裝（目前 sandbox-stable 上沒有安裝）

### Bokeh and HoloViz Panel
* HoloViz Panel 目前（Oct 2020）唯一可以跑遍 widget input combinations 存下結果製造靜態網頁的 framework，然後就可以佈署到 github.io（或 RTD？）
* HoloViz Panel 目前沒有 [drag and drop](https://github.com/holoviz/panel/issues/917)，只有[按紐的 FileInput](https://panel.holoviz.org/reference/widgets/FileInput.html)
* Bokeh 也能做有 widget 的靜態網頁，但需要自己手動先把結果存下來然後用 JavaScript 寫 callback。Plotly Dash 的 callback 就可以用 Python 寫但做出來是動態網頁
* 要做 slideshow 要裝 [RISE](https://github.com/damianavila/RISE)。[FAQ](https://panel.holoviz.org/FAQ.html) 裡有人問
    * 但 RISE 目前為止只能在 Classic Jupyter Notebook 上用，看[這個](https://github.com/damianavila/RISE/issues/270) issue 

### Plotly Dash
* 實測用 [Jupyter Dash](https://github.com/plotly/jupyter-dash) 做的動態網頁無法透過 Voila 佈署到 mybinder 上，看這個 [issue](https://github.com/plotly/jupyter-dash/issues/23)。所以 Jupyter Dash 只能用來在 JupyterLab 裡開發
* ```app.run_server()``` 會暫時佈署到 mybinder 上，可以在開發時測試用
    * 用的是 mybinder 配置給這個 JupyterLab 的 node。把目前的網址 lab 以下取代成 ```proxy/8050``` 就行了。例如
        * ```https://hub.gke2.mybinder.org/user/beginnersc-sandbox-dash-73nkpf4u/lab/workspaces/auto-F?clone=auto-M``` 變成
        * ```https://hub.gke2.mybinder.org/user/beginnersc-sandbox-dash-73nkpf4u/proxy/8050```
    * 如果有 Voila 以外的辦法可以自動 trigger notebook 執行也可以跑 ```app.run_server()``` 生成暫時可用的 Plotly Dash 網頁
* 目前看來還是只能佈署到 Heroku 上
* 只有 Dash Enterprise 才有 auth，不過有人做了 [Flask-Login on Dash App](https://github.com/RafaelMiquelino/dash-flask-login)
* [Jupyter Dash](https://github.com/plotly/jupyter-dash) 和 [jupyter-plotly-dash](https://github.com/GibbsConsulting/jupyter-plotly-dash) 到底有什麼不同？

### Heroku for Deployment
* 可以用來佈署動態/靜態網頁，如 Sphinx documents。不限 framework，不像 Voila 只能呈現 Jupyter notebook 的 output cell
* 可以用 Flask 加密碼保護
* unscalable，訪問量大的話會比 AWS 貴很多，但 AWS 設置起來比 Heroku 複雜多了

## nbsphinx and readthedocs

### Import a Project From GitHub to RTD

* 如果 RTD 不是以 GitHub 註冊的，要先把 GitHub 加到 Connected Services 裡
    * RTD Dashbord > Settings > Connected Services
* RTD Dashboard > Import a Project
* Select a repo
* Input a project name（不能和 RTD 目前上線的任何 project 重名）
* GitHub 端的 webhook 會自動建立，也可以去 RTD 這個 project 裡的 admin > integrations 看

### nbsphinx

* See sphinx tutorial [new](https://www.youtube.com/watch?v=RvJ54ADcVno), [old](https://www.youtube.com/watch?v=oJsUvBQyHBs&feature=youtu.be) & [nbsphinx doc](https://nbsphinx.readthedocs.io/en/0.7.1/index.html)
* 建一個 folder docs
* 進到 docs 裡 ```sphinx-quickstart``` 開啟 wizard
    * [] 裡的是預設值
    * release 隨便打，例如 0.1
* 跑完會出現 docs/source/conf.py，把 ```'nbsphinx'``` 加到 ```extensions``` 裡
* 在 docs 裡 ```make html```，生成的 index.html 在 docs/build/html 裡
* 在 docs/source/index.rst 裡的 toctree 很重要不能 delete 不然 sphinx 會沒辦法 make
* 在 docs/source 裡新增 ipynb，在 index.rst 的 toctree 裡紀錄 ipynb 的檔名（不需要副檔名）
    * 每個 ipynb file 一定要有 title
    * toctree 要對齊像這樣：
        ```
        .. toctree::
           :maxdepth: 2
           :caption: Contents:
                                <---- 這裡一定要有一個空行 
           MYIPYNBFILENAME
        ```
    * 可以加 ```:hidden:``` 然後就會只出現在網頁左邊
    * 參考 [sphinx getting started guide](https://www.sphinx-doc.org/en/master/usage/quickstart.html) & [toctree directive](https://www.sphinx-doc.org/en/master/usage/restructuredtext/directives.html#directive-toctree)
    * 也可以改 toctree maxdepth
    * 可以有多個 toctree 用不同的 captions，像 [JupyterLab doc](https://jupyterlab.readthedocs.io/en/stable/)
* 切換成 3rd party readthedoc theme
    * 需要事先 ```pip install sphinx_rtd_theme```
    * 到 conf.py import 並更換 ```html_theme```：
        ```
        import sphinx_rtd_theme
        html_theme = 'sphinx_rtd_theme'
        ```
* 內容全部加完後回到 docs 裡重新 ```make html```
* ```make clean``` 刪除所有 build 裡的內容
* 要 render bokeh plots 要看這個 [issue](https://github.com/spatialaudio/nbsphinx/issues/61)
* Sphinx 做的網頁目前（Oct 2020）沒辦法在 JupyterLab 正確顯示，[local css 會有問題](https://discourse.jupyter.org/t/loading-static-css-in-jupyterlab/1088/7)


### Build in RTD

* [在 RTD 環境配置裡安裝 nbsphinx](https://nbsphinx.readthedocs.io/en/0.3.3/usage.html#Automatic-Creation-of-HTML-and-PDF-output-on-readthedocs.org)，在這個 repo 裡創建 requirements.txt 加入以下內容：
    ```
    sphinx>=1.4
    ipykernel
    nbsphinx
    ```
* 在 conf.py 裡加入 ```master_doc = 'index'``` 不然會出現 [contents.rst not found Error](https://stackoverflow.com/questions/56336234/build-fail-sphinx-error-contents-rst-not-found)
    * RTD 預設用 ```contents.rst``` 作為 entry point 而非 ```index.rst```
* 在 conf.py 裡加入 ```nbsphinx_allow_errors = True``` 不然用 ipywidgets 時 build 會失敗
* 每次 push 回 GitHub，對應的 RTD project 就會自己重新 build（看 project 裡的 Builds）。也可以自己在 Overview 裡 Build version
* RTD 只需要 source 裡的內容就夠了，所以在 local ```make clean``` 再 push 回 GitHub 也沒問題
* [把這段貼到 conf.py 讓 RTD PDF 編譯中文](https://www.kawabangga.com/posts/2331)（繁體中文需要用字型 [bsmi](https://wlzhong.wordpress.com/2016/10/31/latex-%E5%A6%82%E4%BD%95%E5%9C%A8%E6%96%87%E7%AB%A0%E4%B8%AD%E8%BC%B8%E5%85%A5%E4%B8%AD%E6%96%87/)）
    ```
    latex_elements = {
    # The paper size ('letterpaper' or 'a4paper').
    #'papersize': 'letterpaper',
    #
    # The font size ('10pt', '11pt' or '12pt').
    #'pointsize': '10pt',
    #
    # Additional stuff for the LaTeX preamble.
    #'preamble': '',
    'preamble': r'''
    \hypersetup{unicode=true}
    \usepackage{CJKutf8}
    \DeclareUnicodeCharacter{00A0}{\nobreakspace}
    \DeclareUnicodeCharacter{2203}{\ensuremath{\exists}}
    \DeclareUnicodeCharacter{2200}{\ensuremath{\forall}}
    \DeclareUnicodeCharacter{2286}{\ensuremath{\subseteq}}
    \DeclareUnicodeCharacter{2713}{x}
    \DeclareUnicodeCharacter{27FA}{\ensuremath{\Longleftrightarrow}}
    \DeclareUnicodeCharacter{221A}{\ensuremath{\sqrt{}}}
    \DeclareUnicodeCharacter{221B}{\ensuremath{\sqrt[3]{}}}
    \DeclareUnicodeCharacter{2295}{\ensuremath{\oplus}}
    \DeclareUnicodeCharacter{2297}{\ensuremath{\otimes}}
    \begin{CJK*}{UTF8}{bsmi}
    \AtEndDocument{\end{CJK}}
    ''',
    }
    ```
* RTD 只有 enterprise 才有 private documentation

### [autodoc](https://www.youtube.com/watch?v=LQ6pFgQXQ0Q)（沒試過）

* 去 conf.py 裡 uncomment ```import os, sys``` 和 ```sys.path.insert(0, os.path.abspath('.'))``` 並且把 ```'sphinx.ext.autodoc'``` 加到 extensions 裡
* ```sys.path.insert(0, os.path.abspath('.'))``` 裡面放的是 code 的路徑
* 在 rst 裡加入
    ```
    .. automodule:: MYPYTHONSCRIPTNAME
       :members:
    ```
* 或者用 [sphinx-apidoc](https://www.sphinx-doc.org/en/master/man/sphinx-apidoc.html) 自動掃過所有 script 並產生 rst
    * [過程中會執行所有 script](https://www.youtube.com/watch?v=qrcj7sVuvUA)，所以除了 class，function 和 ```if __name__ == "__main__":``` block 以外不要在 script 裡寫其它東西    
* [sphinx doc](https://www.sphinx-doc.org/en/master/usage/quickstart.html#autodoc)    

## Binder

* 可以用 JupyterLab 打開，JupyterLab 裡也有 internet access 可以用 Terminal push 到 github
* 沒有給 local storage 所以每次改完一定要 commit + push 不然就不見了
* 有一個小缺點就是 inactive 十分鐘 kernel 就會自己斷掉，不過有 offlinenotebook 就沒有影響了

### 環境配置（from [Binder doc](https://mybinder.readthedocs.io/en/latest/config_files.html)）

* 如果只是要加 python package，可以用一個 requirements.txt 列下需要有哪些 python packages，像這樣：
    ```
    numpy
    scipy
    pandas
    ```
* 預設是安裝最新版，也可以指定版本例如 ```seaborn==0.11.0```
* 加 C++ Kernel：
    1. 複製 [xeus-cling repo](https://github.com/jupyter-xeus/xeus-cling) 裡的 environment.yml 放在這裡
    1. 在 dependencies 下把需要的 python package 加進去
    1. 一旦配置了 environment.yml，requirements.txt 會自動被乎略（更 general 的，一旦配置了 Dockerfile，requirements.txt 和 environment.yml 都會自動被乎略）
    1. 更多細節看[這裡](https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html)
* Jupyterlab extension，例如 [toc](https://github.com/jupyterlab/jupyterlab-toc) 要新建在一個 postBuild file 寫
    ```
    jupyter labextension install @jupyterlab/toc
    ```
* [Python debugger](https://github.com/jupyterlab/debugger) 同時需要用 conda 安裝 nodejs 和 xeus-python kernel 和 labextension @jupyterlab/debugger，所以 environment.yml 和 postBuild 都要改
* postBuild
    * postBuild 可以執行任何 bash commands
    * 用 postBuild 安裝 execution time labextension + （用 bash）去 nbextension folder 把這個功能打開的[例子](https://github.com/deshaw/jupyterlab-execute-time/blob/master/binder/postBuild)（這個是 D. E. Shaw 的 repo）
    * 把 git add, commit, push 寫在一個 command 裡並放到 .bashrc 裡，在 postBuild 裡寫下這段：
        ```
        echo '\n# git add commit and push in one command \n' >> .bashrc
        echo 'function cnp() {                                                                  ' >> .bashrc
        echo '    git add *                                                                     ' >> .bashrc
        echo '    git config --global user.name "beginnerSC"                                    ' >> .bashrc
        echo '    git config --global user.email "25188222+beginnerSC@users.noreply.github.com" ' >> .bashrc
        echo '    git commit -a -m "update"                                                     ' >> .bashrc
        echo '    git push                                                                      ' >> .bashrc
        echo '}                                                                                 ' >> .bashrc
        ```
    * 但 push 還是需要敲 GitHub 密碼。如果要避免每次敲密碼應該永遠用 token pull（在 [beginnersc.github.io](https://beginnersc.github.io) 用 "private" 登入）
* 最好不要照著 doc install 隨便改寫 environment.yml 和 postBuild。最穩定的辦法還是去找要安裝的 kernel 有沒有 binder link，照抄那個 repo 裡的配置
* 如果要安裝 latex 沒辦法用 conda 安裝，打開 JupyterLab 再安裝也有權限問題（反正也留不下來），要用 apt.txt，參考 [binder-examples/latex](https://github.com/binder-examples/latex)：
    ```
    cron
    pandoc
    dvipng
    ghostscript
    texlive-fonts-recommended
    texlive-generic-recommended
    texlive-latex-base
    texlive-latex-extra
    texlive-latex-recommended
    texlive-publishers
    texlive-science
    texlive-xetex
    texlive-lang-chinese
    ```
* nbconvert to PDF 編譯中文：
    * 在 apt.txt 裡需要有 texlive-lang-chinese
    * nbconvert 編譯 tex 檔時會用一個 template，要去修改
        * ```find ../../.. -name "article.tplx" | sort```
        * 會找出 ```../../../srv/conda/envs/notebook/lib/python3.7/site-packages/nbconvert/templates/latex/article.tplx```
        * 進到那個 folder 裡用 vim 修改，把 ```\documentclass[11pt]{article}``` 替換為
            ```
            \documentclass{article}
            \usepackage{xeCJK}
            ```
        * 參考[這個 issue](https://github.com/c1mone/Tensorflow-101/issues/4)
        * 實測用 ctex 會把日期也編譯成中文，xeCJK 不會，所以在 template 裡 hard code 永遠用 xeCJK 就可以中英兩種文件都編
        * 有些繁體字沒有字型，例如「複」「裡」會編成亂碼，大扣分！
        * 後來 ```nbconvert/templates``` 不知道為什麼被清空了。只剩下 ```../../../usr/share/texlive/texmf-dist/tex/latex/base/article.sty``` 改這個不知道有沒有用。知道繁體字會編成亂碼也就懶得試了
    * 因為 template 不在 home 之下，要用 postBuild 每次 launch JupyterLab 的時候自動改（去看 sandbox-stable 裡的 postBuild）
* 開啟 Binder 所需時間
    * 每次 commit 之後第一次開會很慢，因為 binder 根據這些 config 生成了 Dockerfile 並且推送到 Docker Hub 上
    * 加了 c++ kernel 和幾個 python packages 之後 rebuild 竟然要半小時！
    * 如果用兩個 repo，一個存內容一個存配置好的環境會快很多（下面有更詳細的說明）
* 雖然不用自己手動配置但 kernel 在 ```../../srv/conda/envs/notebook/share/jupyter/kernels```
* xcpp 相關的執行檔在 ```../../srv/conda/envs/notebook/bin```
* 配置 Dockerfile
    * 創建一個檔案名為 Dockerfile（no extension）放在 sandbox 裡
    * 一旦配置了 Dockerfile，requirements.txt 和 environment.yml 都會自動被乎略
    * [sandbox-latex](https://github.com/beginnerSC/sandbox-latex) 裡的配置可以成功的 Export notebook as PDF。這個配置參考了：
        * [Binder examples 裡的 minimal Dockerfile](https://github.com/binder-examples/minimal-dockerfile)
        * [nbconver installation](https://nbconvert.readthedocs.io/en/latest/install.html) 裡有提到需要哪些 texlive package
        * [JupyterLab installation](https://jupyterlab.readthedocs.io/en/stable/getting_started/installation.html)
        * [latex-online 的 Dockerfile](https://hub.docker.com/r/aslushnikov/latex-online/dockerfile)
            * [Latex.Online](https://latexonline.cc/) 是一個有付 API 的 online latex compiler
            * 更多 latex package 如果有需要可以到這個 Dockerfile 找
        * 目前的配置只是照抄 Binder examples 裡的 minimal 然後把 latex-online Dockerfile 裡的 texlive package 只選需要的裝進去
    * 用同樣的方法也成功灌了 cron（在 pandoc 上面多加一行 cron 就行了）但沒有 editor 所以還是沒辦法 ```crontab -e```
    * 有一個 [Jupyter Docker Stacks](https://jupyter-docker-stacks.readthedocs.io/en/latest/) 裡有很多配好的 Dockerfile 可以參考，學怎麼配置自己要的 Dockerfile（網頁左上有 GitHub 連結）

### 內容和環境分開配置在兩個 repo

* 參考[這篇 post](https://discourse.jupyter.org/t/how-to-reduce-mybinder-org-repository-startup-time/4956) 和作者的 [env repo](https://github.com/choldgraf/binder-sandbox)
* 環境 repo 配置完成之後不需要經常更改，所以沒有每次 commit 完開啟 Binder 就重新 build Dockerfile 的問題
* 內容是用一個 nbgitpuller load 到開機完成的 Binder 裡的
* 所以在 environment.yml 裡需要指定安裝 nbgitpuller，還要在 postBuild 裡 enable（nbgitpuller 是一個 Jupyter server extension）
* content repo 裡不能有 environment.yml 等 config file，不然開一次 env repo 就會壞掉了，重新 commit rebuild 也沒用
* 預設是用 Classic Jupyter Notebook 打開。如果要用 JupyterLab 打開，連結非常複雜，[nbgitpuller 的文檔](https://jupyterhub.github.io/nbgitpuller/link.html)也沒寫清楚，不過[這位老兄](https://edu.oggm.org/en/latest/user_content.html) figure out 了
* 要從 Terminal commit content 的時候要先進到 content repo folder

### nbgitpuller 可以 [pull private project](https://github.com/jupyterhub/nbgitpuller/issues/53)

* 有兩個方法，一個是用 token，缺點是連結裡面會有 token，另一個是用一個 git proxy，看上面的連結討論
* 要用 token 生成連結首先要知道 [git 怎麼用 token 直接 pull private project](https://github.blog/2012-09-21-easier-builds-and-deployments-using-git-over-https-and-oauth/)
* GitHub 登入後 generate 一個 token：Settings -> Developer settings -> Generate new token -> 存在一個安全的地方
    * token 不分 project，所以知道這個 token 的人就有所有 private project 的讀寫權限
    * 實測過用 token 打開的 JupyterLab 在 git push 的時候不需要再敲一次帳號密碼
* 最終 JupyterLab fast startup 連結是這樣的：https://mybinder.org/v2/gh/beginnerSC/sandbox-stable/master?urlpath=git-pull?repo=https://___TOKEN___@github.com/beginnerSC/___PRIVATE_PROJECT___%26amp%3Bbranch=master%26amp%3Burlpath=lab/tree/___PRIVATE_PROJECT___?autodecode

## Cloud JupyterLab Solutions Other Than Binder

* Notebooks.ai 掛掉之後只剩下[一些選項](https://www.dataschool.io/cloud-services-for-jupyter-notebook/?fbclid=IwAR316JuoHek2bAFjYEgvFH2XIJDtUMoaBNKCml7nusaZkOB0oTTKSnymeu0)
* 免費的選項中最好的是 Binder，只是要學會配置環境
* 其實 Binder 不是設計來這樣用的，最好還是研究怎麼在 Google Colab 上安裝 JupyterLab
* private project solution: 
    * CoCalc
        * 可以用 JupyterLab 打開（打開 settings 之後在最右下角）
        * 連 github 或 publish 或 pip install 需要取得 internet access，都是付費才能打開的功能
        * 但在 cocalc 本地端 Terminal 用 git 是可以的
        * 唯一的問題就是哪天 cocalc 如果又要倒，檔案只能一個一個下載（或者想辦法下載 .git 檔就可以了？）
* public project solutions: 
    * Gitpod
        * 直接在一個 github repo 的網址列前面加上 ```https://gitpod.io/#``` 打開項目
        * ipynb file 上按右鍵可以選 Open in Notebook Editor，編輯完在左邊 stage + commit 然後到右邊 push 到 github 上。但這個編輯器沒有 JupyterLab 好用
        * free account 每個月有 50 小時的使用時間限制
        * 付費會員可以編輯使用時間變成每個月 100 小時並且可以開 private github projects
        * gitpod 在打開 project 的時候會檢查此 project 是否 public，編輯完 commit 的時候不會
    * Kyso (read-only)
        * 像 Blog，讀者可以留言。Kyso 可以 render nb（nicely），也可以連 github public/private projects 還自動同步，但不能直接在 Kyso 編輯
        * 實測過和 GitHub 是同步的
            * make sense 因為有 webhook（GitHub Repo -> Settings -> Webhooks），commit 的時候會自動觸發 Kyso 同步
        * authorize 的時候要選 all repo 才能連 private project
        * Kyso 老記不住哪一個是 main file（然後就沒有 Files tab 可以按，也看不到任何 notebook），加 kyso.yaml 也沒用，README 裡的連結是最後是直接貼 README 在 Kyso 的連結才解決的
            * 進去之後自己進 Files tab 看所有 notebook
        * Code Hidden/Shown
            * 所有 nb 打開的時候預設是 Code Hidden，所有 input cell 都看不見。右上角可以選 Code Shown
            * 像這樣貼連結給人，對方打開的時候才是看的到 code 的：https://kyso.io/beginnerSC/misc/file/Piano.ipynb#code=shown
        * Kyso 沒辦法顯示 raw cell 不過本來就應該盡量避免 raw cell。nbviewer 能顯示可是會亂掉
        * 還是需要由 nbviewer 補足因為：
            * 手機平板上無法顯示 Files tab 也無法顯示 Code Hidden/Shown 的選單，這樣就沒辦法從 README 的連結找到其它 nb 了
            * 直接在 input cell 貼上的圖沒辦法顯示（GitHub 也不行），例如 [Backprop.ipynb](https://nbviewer.jupyter.org/github/beginnerSC/misc/blob/master/Backprop.ipynb) 的圖，只有在 nbviewer 看的到
            * 一張圖如果要在 Kyso 看的見就要存成圖檔，像 [Theory.ipynb](https://nbviewer.jupyter.org/github/beginnerSC/misc/blob/master/Theory.ipynb)
    * nbviewer (read-only)
        * 缺點不是即時的，據說有 10 分鐘 delay（真的只有 10 分鐘嗎？）可以手動在 url 後面加上 ```?flush_cache=true``` 或 ```?flush_cache=True``` 有時候會有用（browser dependent）
        * readme 裡的 url 是
            * nbviewer：https://nbviewer.jupyter.org/github/beginnerSC/misc/tree/master/
            * Kyso：https://kyso.io/beginnerSC/misc/file/README.md
            * Gitpod：https://gitpod.io/#https://github.com/beginnerSC/misc
            * JupyterLab (Binder, launch from this repo)：https://mybinder.org/v2/gh/beginnerSC/misc/master?urlpath=lab
                * 把 JupyterLab url 最後面的 ```?urlpath=lab``` 拿掉就變成普通的 binder link，用 Jupyter Notebook 打開
                * [這裡](https://github.com/binder-examples/jupyterlab)有說明
                * 這個 repo 的 environment.yml 已經砍掉了。可以還是開但連 numpy 都沒有
            * Classic Jupyter Notebook (fast startup)：https://mybinder.org/v2/gh/beginnerSC/sandbox/master?urlpath=git-pull?repo=https://github.com/beginnerSC/misc
            * JupyterLab (fast startup)：https://mybinder.org/v2/gh/beginnerSC/sandbox/master?urlpath=git-pull?repo=https://github.com/beginnerSC/misc%26amp%3Bbranch=master%26amp%3Burlpath=lab/tree/misc?Fautodecode
                * 那個 %26amp%3B 是 & 的 encoding 但網址列認不得所以還是只能寫 %26amp%3B
* public/private project solution
    * Code Ocean 
        * 可以用 JupyterLab 打開，JupyterLab 裡也有 internet access 可以用 Terminal push 到 github
        * 在 env 裡可以自己 install 需要的 package 如 numpy，scipy，pandas
        * 只能 import from public github repo（如果要 import private repo 要打開一瞬間）
        * 唯一的缺點是每個月十小時的計算時間限制
    * Binder
            
        